In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler

In [3]:
df= pd.read_csv('data.csv')
df.head()

,Part filling degree,PartRate wl,PartRate hl,PartRate wh,Part height,Part Bottom area,Total filling degree,TotalRate wl,TotalRate hl,TotalRate wh,Bottom area,Heigh,NumPart
0,8.996,0.100,0.079,1.267,48.395,1286.173,8.280,0.541,0.450,1.202,2585.639,311.0000,45
1,20.317,1.294,0.673,1.923,59.242,468.536,17.022,0.553,0.264,2.096,2480.893,176.7000,59
2,25.118,1.211,0.366,3.310,35.281,548.913,15.891,0.538,0.428,1.258,2581.136,296.4183,115
3,9.129,1.211,0.436,2.779,82.302,1012.383,6.892,0.528,0.528,1.000,2508.800,363.7444,41
4,10.079,0.835,0.279,2.989,29.148,315.591,10.541,0.536,0.113,4.753,2477.000,76.6857,27


In [4]:
df.shape

(104, 13)

In [5]:
df.columns

Index(['Part filling degree', 'PartRate wl', 'PartRate hl', 'PartRate wh',
       'Part height', 'Part Bottom area', 'Total filling degree',
       'TotalRate wl', 'TotalRate hl', 'TotalRate wh', 'Bottom area', 'Heigh ',
       'NumPart'],
      dtype='object')

In [6]:
# Split data in x and y
x= df[['PartRate wl', 'PartRate hl', 'PartRate wh',
       'Part height', 'Part Bottom area', 'Total filling degree',
       'TotalRate wl', 'TotalRate hl', 'TotalRate wh', 'Bottom area', 'Heigh ']]
y= df[['Part filling degree']]

In [7]:
# Split data into train and test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state=1)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(83, 11) (21, 11) (83, 1) (21, 1)


## GP

In [8]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

gp= GaussianProcessRegressor(alpha=0.1, n_restarts_optimizer=20)
gp.fit(x_train, y_train)

GaussianProcessRegressor(alpha=0.1, n_restarts_optimizer=20)

In [9]:
y_train_gp= gp.predict(x_train)
y_test_gp= gp.predict(x_test)

In [10]:
# Mean squared erro
from sklearn.metrics import mean_squared_error
mse_train_gp= mean_squared_error(y_train, y_train_gp)
mse_test_gp= mean_squared_error(y_test, y_test_gp)
print(mse_train_gp)
print('............')
print(mse_test_gp)

2.9866766307876227
............
231.599473


In [11]:
# Function defination for R2 and Adj R2

def r2(actual: np.ndarray, predicted: np.ndarray):
    """ R2 Score """
    return r2_score(actual, predicted)

def adjr2(actual: np.ndarray, predicted: np.ndarray, rowcount: np.int, featurecount: np.int):
    """ Adj R2 Score """
    return 1-(1-r2(actual,predicted))*(rowcount-1)/(rowcount-featurecount)

In [12]:
from sklearn.metrics import r2_score
r2_train_gp= r2(y_train, y_train_gp)
r2_test_gp= r2(y_test, y_test_gp)

adj_r2_train_gp= adjr2(y_train, y_train_gp, 60, 2)
adj_r2_test_gp= adjr2(y_test, y_test_gp, 15, 2)

print(r2_train_gp)
print('............')
print(r2_test_gp)
print('............')
print(adj_r2_train_gp)
print('............')
print(adj_r2_test_gp)

0.9644804909499066
............
-2.4342217484884197
............
0.9638680856214568
............
-2.698392652218298


In [13]:
# Mean absolute error
from sklearn.metrics import mean_absolute_error
mae_train_gp= mean_absolute_error(y_train, y_train_gp)
mae_test_gp= mean_absolute_error(y_test, y_test_gp)
print(mae_train_gp)
print('............')
print(mae_test_gp)

1.5138543263964974
............
12.812523809523809


In [14]:
# RMSE
rmse_train_gp = np.sqrt(mse_train_gp)
rmse_test_gp = np.sqrt(mse_test_gp)
print(rmse_train_gp)
print('............')
print(rmse_test_gp)

1.7282004023803554
............
15.21839258923228


In [15]:
#Randomized search
# initialize model and define the space of the hyperparameters to
# perform the grid-search over
model = gp
n_restarts_optimizer = [5, 10, 20, 40]
alpha = [0.001, 0.01, 0.1, 1]


grid = dict(n_restarts_optimizer=n_restarts_optimizer, alpha=alpha)

In [16]:
# initialize a cross-validation fold and perform a randomized-search
# to tune the hyperparameters
print("[INFO] grid searching over the hyperparameters...")
cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1234)
randomSearch = RandomizedSearchCV(estimator=model, n_jobs=-1,
	cv=cvFold, param_distributions=grid,
	scoring="neg_mean_squared_error")
searchResults = randomSearch.fit(x_train, y_train)
# extract the best model and evaluate it
print("[INFO] evaluating...")
bestModel = searchResults.best_estimator_
print("R2: {:.2f}".format(bestModel.score(x_test, y_test)))
print(bestModel)

[INFO] grid searching over the hyperparameters...
[INFO] evaluating...
R2: -2.43
GaussianProcessRegressor(alpha=1, n_restarts_optimizer=40)


## Lazipredict

In [17]:
from lazypredict.Supervised import LazyRegressor

clf = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(x_train, x_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 38.89it/s]

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
LinearSVR                                    0.04       0.57  5.39        0.01
HuberRegressor                              -0.05       0.53  5.65        0.01
GradientBoostingRegressor                   -0.11       0.50  5.80        0.04
KNeighborsRegressor                         -0.23       0.45  6.10        0.01
Lars                                        -0.23       0.44  6.12        0.01
LinearRegression                            -0.23       0.44  6.12        0.01
TransformedTargetRegressor                  -0.23       0.44  6.12        0.01
SGDRegressor                                -0.30       0.42  6.27        0.01
LassoLarsIC                                 -0.30       0.41  6.28        0.01
Ridge                                       -0.32       0.40  6.34        0.01
LassoCV                                     -0.34   

## XGB

In [18]:
from xgboost import XGBRegressor
xgb= XGBRegressor()
xgb.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [19]:
y_train_xgb= xgb.predict(x_train)
y_test_xgb= xgb.predict(x_test)

In [20]:
r2_train_xgb= r2(y_train, y_train_xgb)
r2_test_xgb= r2(y_test, y_test_xgb)

adj_r2_train_xgb= adjr2(y_train, y_train_xgb, 60, 2)
adj_r2_test_xgb= adjr2(y_test, y_test_xgb, 15, 2)

print(r2_train_xgb)
print('............')
print(r2_test_xgb)
print('............')
print(adj_r2_train_xgb)
print('............')
print(adj_r2_test_xgb)

0.9999999933211046
............
0.14753339488110462
............
0.9999999932059512
............
0.08195904064118953


In [21]:
# Mean squared error
from sklearn.metrics import mean_squared_error
mse_train_xgb= mean_squared_error(y_train, y_train_xgb)
mse_test_xgb= mean_squared_error(y_test, y_test_xgb)
print(mse_train_xgb)
print('............')
print(mse_test_xgb)

5.615984397128912e-07
............
57.48924529481385


In [22]:
# Mean absolute error
from sklearn.metrics import mean_absolute_error
mae_train_xgb= mean_absolute_error(y_train, y_train_xgb)
mae_test_xgb= mean_absolute_error(y_test, y_test_xgb)
print(mae_train_xgb)
print('............')
print(mae_test_xgb)

0.0005415231865572117
............
5.738007826123918


In [23]:
# RMSE
rmse_train_xgb = np.sqrt(mse_train_xgb)
rmse_test_xgb = np.sqrt(mse_test_xgb)
print(rmse_train_xgb)
print('............')
print(rmse_test_xgb)

0.0007493987187825258
............
7.582166266629468


In [24]:
# initialize model and define the space of the hyperparameters to
# perform the grid-search over
model = xgb
n_estimators = [10, 100, 1000, 10000]
learning_rate = [0.1, 0.01, 1e-3]
max_depth = [5, 10, 20, 40]
reg_lambda = [0.01, 0.1, 1]


grid = dict(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth )

In [25]:
# initialize a cross-validation fold and perform a randomized-search
# to tune the hyperparameters
print("[INFO] grid searching over the hyperparameters...")
cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
randomSearch = RandomizedSearchCV(estimator=model, n_jobs=-1,
	cv=cvFold, param_distributions=grid,
	scoring="neg_mean_squared_error")
searchResults = randomSearch.fit(x_train, y_train)
# extract the best model and evaluate it
print("[INFO] evaluating...")
bestModel = searchResults.best_estimator_
#print("R2: {:.2f}".format(bestModel.score(x_test, y_test)))
print(bestModel)

[INFO] grid searching over the hyperparameters...
[INFO] evaluating...
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)


In [26]:
# Feature importance analysis

importance = xgb.feature_importances_
# summarize feature importance

for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.02358
Feature: 1, Score: 0.01036
Feature: 2, Score: 0.08540
Feature: 3, Score: 0.06386
Feature: 4, Score: 0.29113
Feature: 5, Score: 0.16710
Feature: 6, Score: 0.10547
Feature: 7, Score: 0.09008
Feature: 8, Score: 0.01294
Feature: 9, Score: 0.09548
Feature: 10, Score: 0.05461


## ANN

In [35]:
from sklearn.neural_network import MLPRegressor

mlp= MLPRegressor(activation='tanh', hidden_layer_sizes=10, solver='lbfgs')
mlp.fit(x_train, y_train)

MLPRegressor(activation='tanh', hidden_layer_sizes=10, solver='lbfgs')

In [36]:
y_train_mlp= mlp.predict(x_train)
y_test_mlp= mlp.predict(x_test)

In [37]:
r2_train_mlp= r2(y_train, y_train_mlp)
r2_test_mlp= r2(y_test, y_test_mlp)

adj_r2_train_mlp= adjr2(y_train, y_train_mlp, 60, 2)
adj_r2_test_mlp= adjr2(y_test, y_test_mlp, 15, 2)

print(r2_train_mlp)
print('............')
print(r2_test_mlp)
print('............')
print(adj_r2_train_mlp)
print('............')
print(adj_r2_test_mlp)

0.10036155716013162
............
-0.16333004301646437
............
0.08485054952496152
............
-0.2528169694023461


In [38]:
# Mean squared error
from sklearn.metrics import mean_squared_error
mse_train_mlp= mean_squared_error(y_train, y_train_mlp)
mse_test_mlp= mean_squared_error(y_test, y_test_mlp)
print(mse_train_mlp)
print('............')
print(mse_test_mlp)

75.64657241175854
............
78.45347348530106


In [39]:
# RMSE
rmse_train_mlp = np.sqrt(mse_train_mlp)
rmse_test_mlp = np.sqrt(mse_test_mlp)
print(rmse_train_mlp)
print('............')
print(rmse_test_mlp)

8.697503803492042
............
8.85739654104416


In [32]:
# Mean absolute error
from sklearn.metrics import mean_absolute_error
mae_train_mlp= mean_absolute_error(y_train, y_train_mlp)
mae_test_mlp= mean_absolute_error(y_test, y_test_mlp)
print(mae_train_mlp)
print('............')
print(mae_test_mlp)

29.6032740319461
............
31.480716038359283


In [33]:
# initialize model and define the space of the hyperparameters to
# perform the grid-search over
model = mlp
hidden_layer_sizes = [5, 8, 10]
activation = ['relu','tanh']
solver = ["lbfgs", "sgd", "adam"]

grid = dict(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver )

In [34]:
# initialize a cross-validation fold and perform a randomized-search
# to tune the hyperparameters
print("[INFO] grid searching over the hyperparameters...")
cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1234)
randomSearch = RandomizedSearchCV(estimator=model, n_jobs=-1,
	cv=cvFold, param_distributions=grid,
	scoring="neg_mean_squared_error")
searchResults = randomSearch.fit(x_train, y_train)
# extract the best model and evaluate it
print("[INFO] evaluating...")
bestModel = searchResults.best_estimator_
print("R2: {:.2f}".format(bestModel.score(x_test, y_test)))
print(bestModel)

[INFO] grid searching over the hyperparameters...
[INFO] evaluating...
R2: -0.19
MLPRegressor(activation='tanh', hidden_layer_sizes=10, solver='lbfgs')


In [40]:
## Cross validation
from sklearn.model_selection import cross_val_score as CV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

models = [ xgb,  gp, mlp, ]
labels = ['XGBR', 'GPs', 'MLP', '']
IMS = []

print("--------------------------------------------------------------")
print("%5s | %5s | %5s | %5s | %5s " % ("ML algorithm",  "CV-R2", "CV-MSE", "CV-MAE", "IM"))
print("--------------------------------------------------------------")

for mod, label in zip( models, labels): 
  y_star_train = mod.predict(x_train)
 
  scoreR2 = CV(mod, x_train, y_train, cv=5, scoring='r2')
  #print(-1*scoreR2)
  scoreMSE = CV(mod, x_train, y_train, cv=5, scoring='neg_mean_squared_error')
  #print(-1*scoreMSE)
  scoreMAE = CV(mod, x_train, y_train, cv=5, scoring='neg_mean_absolute_error')
  #print(-1*scoreMAE)
  IM=np.sqrt(((1-scoreR2)**2) + (scoreMSE) + (scoreMAE**2) )
  #print("IM: ", IM)
  
 
  print("%15s | %.4f | %.4f | %.4f | %.4f " % (label,  scoreR2.mean(), -1*scoreMSE.mean(), -1*scoreMAE.mean() , np.sqrt(((1-scoreR2.mean())**2) + (-1*scoreMSE.mean()) + (scoreMAE.mean()**2) )))
print("--------------------------------------------------------------")

--------------------------------------------------------------
ML algorithm | CV-R2 | CV-MSE | CV-MAE |    IM 
--------------------------------------------------------------
           XGBR | 0.0639 | 70.8737 | 6.4418 | 10.6418 
            GPs | -3.5661 | 359.1851 | 16.5967 | 25.6024 
            MLP | -0.0442 | 75.8957 | 7.4769 | 11.5278 
--------------------------------------------------------------


In [41]:
# Prediction of ME

y_ME_gp= gp.predict(x)
df['ME GP predicted']=y_ME_gp

y_ME_xgb= xgb.predict(x)
df['ME XGB predicted']=y_ME_xgb

y_ME_mlp= gp.predict(x)
df['ME MLP predicted']=y_ME_mlp

df

,Part filling degree,PartRate wl,PartRate hl,PartRate wh,Part height,Part Bottom area,Total filling degree,TotalRate wl,TotalRate hl,TotalRate wh,Bottom area,Heigh,NumPart,ME GP predicted,ME XGB predicted,ME MLP predicted
0,9.00,0.10,0.08,1.27,48.40,1286.17,8.28,0.54,0.45,1.20,2585.64,311.00,45,8.18,9.00,8.18
1,20.32,1.29,0.67,1.92,59.24,468.54,17.02,0.55,0.26,2.10,2480.89,176.70,59,18.47,20.32,18.47
2,25.12,1.21,0.37,3.31,35.28,548.91,15.89,0.54,0.43,1.26,2581.14,296.42,115,22.83,25.12,22.83
3,9.13,1.21,0.44,2.78,82.30,1012.38,6.89,0.53,0.53,1.00,2508.80,363.74,41,8.30,9.13,8.30
4,10.08,0.83,0.28,2.99,29.15,315.59,10.54,0.54,0.11,4.75,2477.00,76.69,27,9.16,10.08,9.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,28.50,1.16,0.32,3.65,27.25,335.29,15.16,0.52,0.15,3.39,2316.88,102.27,46,25.91,28.50,25.91
100,28.57,1.21,0.51,2.37,27.99,165.78,13.37,0.53,0.12,4.49,2102.16,74.37,67,25.97,28.57,25.97
101,5.46,0.98,0.18,5.57,44.03,2088.24,4.91,0.54,0.64,0.84,2613.22,444.48,33,0.00,9.70,0.00
102,11.23,0.33,0.16,2.13,63.72,1329.32,5.06,0.54,0.47,1.16,2587.45,322.40,16,10.21,11.23,10.21


In [42]:
df.to_csv('ME_pred.csv')

### Conclusion
- XGBRegressor performs better than GP and ANN.